In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from matplotlib import pyplot    
from PIL import Image
from skimage.io import imread, imsave

import zmq
import numpy as np
import cv2
import io
import timeit

In [ ]:
# Parameters to change !!!

generatorPath = 'generator_100.h5'
socketBind = 'tcp://*:5555'

In [ ]:
model = load_model(generatorPath)

context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind(socketBind)

In [ ]:
# load an image
def load_image(pixels):
    # load image with the preferred size
    # convert to numpy array
    pixels = img_to_array(pixels)
    # scale from [0,255] to [-1,1]
    pixels = (pixels - 127.5) / 127.5
    # reshape to 1 sample
    pixels = np.expand_dims(pixels, 0)
    return pixels

In [ ]:
while True:
    bytes_received = socket.recv()
    print('Received sketch. Processing...')
    start = timeit.default_timer()
    image = Image.open(io.BytesIO(bytes_received))
    image = load_image(image)
    gen_image = model.predict(image)
    # scale from [-1,1] to [0,1]
    gen_image = (gen_image + 1) / 2.0    

    imsave('predicted_map.png', (gen_image[0] * 255.0).astype(np.uint8))
    predicted_map = imread('predicted_map.png')
    predicted_map = np.rot90(predicted_map, 3)
        
    bytes_to_send = predicted_map.tobytes()
    socket.send(bytes_to_send)
    stop = timeit.default_timer()
    print('Created heightmap. Time [s]: ', stop - start) 